# Table of contents

- Download latest data
- Retrain model
- Display current squad
- Use model to identify transfers
- Identify playing 11
- Show top performers by position

In [1]:
import pandas as pd
import torch
import numpy as np
import random
from random import shuffle
np.random.seed(17)
random.seed(17)
torch.manual_seed(17)
import wget
import os
import aiohttp
import asyncio
from fpl import FPL
from torch.utils.data import TensorDataset, DataLoader
from player import Player
from team import Team
from data_processor import get_fpl, get_current_squad
from agent import Agent
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl


## Download latest data, train model and save model.

In [2]:
model_type = "linear"
features = ["total_points", "ict_index", "clean_sheets", "saves", "assists", "was_home","goals_scored"]
logger = pl.loggers.TensorBoardLogger(f"lightning_logs/model_comparison/{model_type}")
trainer = pl.Trainer(max_epochs=50, gpus=torch.cuda.device_count(), logger=logger,  callbacks=[EarlyStopping(monitor="val_loss")])
agent = Agent(features, epochs=50, window=6, model_type=model_type)
await agent.get_data()
await agent.update_model(trainer)

GPU available: True, used: True


TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs


/home/ajrfhp/anaconda3/envs/test/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/home/ajrfhp/Control/Control_FPL/data_processor.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_features["total_points"] = all_player_features["total_points"].clip(0, max_player_points)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Validating:  25%|██████████████████▎                                                      | 1/4 [00:00<00:01,  1.85it/s]

Epoch 0: 100%|█████████████████████████████████████████████████████| 18/18 [00:01<00:00, 13.22it/s, loss=2.31, v_num=12]

Epoch 0:   0%|                                                   | 0/18 [00:00<00:00, 26546.23it/s, loss=2.31, v_num=12]

Epoch 1:   0%|                                                    | 0/18 [00:00<00:00, 1036.65it/s, loss=2.31, v_num=12]

Epoch 1:   0%|                                                      | 0/18 [00:21<06:34, 21.92s/it, loss=2.31, v_num=12]

In [ ]:
current_squad, non_squad = await get_current_squad(features, window=agent.window, num_players=agent.num_players)
current_squad, non_squad = await agent.get_new_squad(features)
squad = current_squad + non_squad


In [ ]:
agent.get_wildcard_squad(squad, max_weight=1000, visualize=True)

## Show game week id

In [ ]:
os.environ['GAMEWEEK']

## Display current squad

In [ ]:
current_squad, non_squad = await get_current_squad(features, window=agent.window, num_players=agent.num_players)
for player in current_squad:
    player.visualize()

## Use model to swap players

In [ ]:
current_squad, non_squad = await agent.get_new_squad(features)

## Set playing 11

In [ ]:
agent.set_playing_11(current_squad, visualize=True)

## Highlight top performers by position for this week

In [ ]:
agent.show_top_performers(current_squad + non_squad, k=5)